In [18]:
import pandas as pd
from typing import List, Any, NamedTuple
import datetime
import re

In [105]:
plik1 = r"LINK_DO_PLIKU"
from bs4 import BeautifulSoup
with open(plik1, "r", encoding='utf-8') as f:
    soup = BeautifulSoup(f)
    lista1 = soup.findAll('div', attrs={'class':'pam _3-95 _2pi0 _2lej uiBoxWhite noborder'})
    print(len(lista1))

1907


In [107]:
calosc = lista1[1: ]

In [69]:
class Message(NamedTuple):
    date_time: datetime.datetime
    message: str
    author: str
    reactions: str

    def __repr__(self):
        return f"Author: {self.author}\n" \
               f"Message: {self.message}\n" \
               f"Reactions: {self.reactions}\n" \
               f"Date: {self.date_time}"

In [170]:
class Row:
    message: Message
    def __init__(self, row):
        self.message = self.all_information(row)

    def all_information(self, row) -> Message:
        date_time = self._get_date(row)
        message = self._get_message(row)
        author = self._get_author(row)
        reactions = self._get_reactions(row)
        return Message(date_time=date_time,
                       message=message,
                       author=author,
                       reactions=reactions)

    def _get_message(self, row) -> str:
        if 'href' in str(row):
            return 'IMAGE'
        mess_ = row.find('div', attrs={'class':'_3-96 _2let'}).text
        mess = ''.join([x
                        if x.isidentifier() or x.isascii() else '--'
                        for x in mess_]).split('--')[0]
        return mess

    def _get_author(self, row) -> str:
        return row.find('div', attrs={'class':'_3-96 _2pio _2lek _2lel'}).text

    def _get_reactions(self, row) -> str:
        found = row.find('ul', attrs={'class':'_tqp'})
        if found:
            return '-'.join([x.text for x in found.findAll('li')])
        else:
            return ''

    def _get_date(self, row) -> datetime.datetime:
        week_days = {"Niedziela": 0, "Poniedziałek":1,"Wtorek":2,"Środa":3,"Czwartek":4,"Piątek":5,"Sobota":6}
        months = {"cze":6,"maj":5,"kwi":4,"mar":3,"lut":2,"sty":1,"gru":12,"lis":11,"paź":10, "wrz": 9, "lip":7, "sie":8}
        row_source = row.find('div', attrs={'class':'_3-94 _2lem'}).text
        row_source = row_source.replace(',', '')
        # 'sty 30, 2021 1:56:19po południu'
        date_from = row_source.split(' ')
        # ['sty', '30,', '2021', '1:56:19po', 'południu']
        pattern = r'\d+\:\d+\:\d+'
        time = re.findall(pattern, row_source)[0]
        # 'sty 30, 2021 1:56:19po południu' || ['1:56:19']
        hour = time.split(':')[0]
        minute = time.split(':')[1]
        if row_source[-1] == 'u':
            hour = int(hour)
            hour += 12
        if hour == 24:
            hour = 0
        return datetime.datetime(year=int(date_from[2]),
                                 month=int(months[date_from[0]]),
                                 day=int(date_from[1]),
                                 hour=int(hour),
                                 minute=int(minute))

    def __repr__(self) -> str:
        return str(self.message)

In [176]:
def create_CSV(name: str) -> None:
    try:
        df = pd.DataFrame({}, columns=["Author", "Message", "Reactions", "Date"])
        for row in calosc:
            row_ = Row(row)
            df.loc[len(df)] = [row_.message.author] \
                              + [row_.message.message] \
                              + [row_.message.reactions] \
                              + [row_.message.date_time]
        df = df.sort_values('Date')
        df.to_csv(f'{name}.csv', index=False, encoding='utf-8-sig')
        print("Saved")
    except Exception:
        print("[ERROR] Not Saved")

In [177]:
create_CSV('WszystkoZapisane')

Saved
